In [ ]:
# 폰트 설치
!sudo apt-get install -y fonts-nanum    
!sudo fc-cache -fv                      
!rm ~/.cache/matplotlib -rf

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_path = '/usr/share/fonts/truetype/nanum/NanumMyeongjo.ttf' 
font_name  = fm.FontProperties(fname=font_path, size=12).get_name( )
plt.rcParams['font.family'] = font_name
fm._rebuild( )
%config  InlineBackend.figure_format = "retina"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import re
warnings.filterwarnings('ignore')
from tqdm.auto import tqdm

In [ ]:
# LDA
!pip install -U gensim
import gensim
from gensim import corpora

In [ ]:
# LDA 시각화
!pip install pyLDAvis
import pyLDAvis.gensim_models

In [ ]:
album_df = pd.read_csv(dirpath+"album.csv", index_col=[0])
song_df = pd.read_csv(dirpath+"song.csv", index_col=[0])

In [ ]:
import ast
song_df['vocabs'] = song_df['vocabs'].apply(lambda x : ast.literal_eval(x))

In [ ]:
song_df

,album_id,song_id,song_title,lyrics,genre,vocabs
Unnamed: 0,,,,,,
0,10827816,34481680,드라마,나도 한때는 그이의 손을 잡고 내가 온 세상 주인공이 된 듯 꽃송이의 꽃잎 하나하나...,발라드,"[그이, 세상, 주인공, 꽃송이, 꽃잎, 하나하나, 모두, 위하, 올림픽대로, 뚝섬..."
1,10827816,34481681,정거장,다음 정거장에서 만나게 될까 그리워했던 얼굴을 다음 파란불에는 만나게 될까 그리곤 ...,포크/블루스,"[다음, 정거장, 만나, 그립, 얼굴, 다음, 파란불, 만나, 그리, 풍경, 정하,..."
2,10827816,34481682,겨울잠,때 이른 봄 몇 송이 꺾어다 너의 방 문 앞에 두었어 긴 잠 실컷 자고 나오면 그때...,발라드,"[이르, 송이, 실컷, 그때쯤, 예쁘, 띄우, 여름, 따르, 머리맡, 금세, 버리,..."
3,10827816,34481683,너,아득히 떨어진 곳에서 아무 관계없는 것들을 보며 조금 쓸쓸한 기분으로 나는 너를 보...,포크/블루스,"[아득히, 아무, 관계, 것들, 조금, 쓸쓸, 기분, 보고픈, 떠올리, 애달프, 일..."
4,10827816,34481684,러브레터,골목길 머뭇하던 첫 안녕을 기억하오 그날의 끄덕임을 난 잊을 수 없다오 길가에 내...,발라드,"[골목길, 머뭇하, 안녕, 기억하오, 그날, 끄덕이, 길가, 내리, 새벽, 고요, ..."
...,...,...,...,...,...,...
122,398025,1929323,미운오리,미운오리 저기 멀리 어둔 하늘 보다가 엄마 별이 사는 그 곳은 어딜까 외로운 밤 ...,발라드,"[미운오리, 저기, 멀리, 어둔, 하늘, 엄마, 어딜까, 외롭, 날개짓, 오리, 그..."
123,398025,1929324,미아,우리 둘 담아 준 사진을 태워 하나 둘 모아 둔 기억을 지워 그만 일어나 가야 하는...,발라드,"[우리, 사진, 태우, 하나, 모으, 기억, 일어나, 너무, 주저, 빗물, 한치, ..."
124,398025,1929325,있잖아 (Feat. 마리오),Ye this song is my love story 첫 눈에 반한다는 말 절대로 ...,댄스,"[반하, 절대로, 나타나, 순간, 뺏기, 버리, 멈추, 버리, 시간, 지나, 만나,..."


In [ ]:
song_df['temp'] = song_df['vocabs'].apply(lambda x : " ".join(x))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features= 300, max_df = 0.7)
tfidf = vectorizer.fit_transform(song_df['temp'])

print(tfidf.shape)

(122, 300)


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=5, learning_method='online', random_state=1, max_iter=20)
lda_top=lda_model.fit_transform(tfidf)

In [ ]:
lda_model.components_.shape

(5, 300)

In [ ]:
terms = vectorizer.get_feature_names_out() 
def get_topics(components, feature_names, n=10):
    for index, topic in enumerate(components):
        print("Topic %d:" % (index+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n-1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('사랑', 4.38), ('사람', 4.02), ('그대', 3.63), ('마음', 3.49), ('모르', 2.96), ('조금', 2.93), ('아니', 2.81), ('기다리', 2.72), ('그렇', 2.62), ('아이', 2.56)]
Topic 2: [('차라리', 1.24), ('삼촌', 1.1), ('어젯밤', 1.09), ('그리', 1.09), ('바람', 1.05), ('소리', 0.93), ('만들', 0.92), ('보이', 0.91), ('내겐', 0.9), ('삐에로', 0.89)]
Topic 3: [('원하', 1.51), ('사랑한단', 1.41), ('사랑', 1.33), ('마디', 1.31), ('것이길', 1.31), ('솔직히', 1.28), ('전부', 1.23), ('정말', 1.2), ('여기', 1.12), ('빨리', 1.11)]
Topic 4: [('그대', 2.84), ('사랑', 1.67), ('매일', 1.65), ('아무', 1.64), ('비밀', 1.57), ('모르', 1.56), ('채플린', 1.4), ('누구', 1.31), ('다음', 1.29), ('아니', 1.23)]
Topic 5: [('다시', 2.58), ('노래', 1.97), ('우울', 1.84), ('모으', 1.57), ('바라', 1.47), ('기도', 1.42), ('흐르', 1.4), ('하나', 1.39), ('그치', 1.36), ('기억', 1.29)]


In [ ]:
dictionary = corpora.Dictionary(song_df['vocabs'])
corpus = [dictionary.doc2bow(v) for v in song_df['vocabs']]

In [ ]:
len(dictionary)

1923

In [ ]:
TOPIC_NUM = 5
LDA_model = gensim.models.ldamodel.LdaModel(corpus, num_topics = TOPIC_NUM, id2word=dictionary, passes=30)

In [ ]:
topics = LDA_model.print_topics(num_words=10)
for t in topics :
  print(t)

(0, '0.021*"사람" + 0.015*"우울" + 0.013*"시간" + 0.012*"모르" + 0.011*"마음" + 0.009*"어디" + 0.008*"아침" + 0.008*"흐르" + 0.007*"그렇" + 0.007*"만나"')
(1, '0.031*"사랑" + 0.014*"하나" + 0.013*"버리" + 0.012*"그대" + 0.011*"사랑한단" + 0.009*"바라" + 0.009*"원하" + 0.008*"마디" + 0.008*"만나" + 0.008*"우리"')
(2, '0.024*"그대" + 0.009*"지금" + 0.009*"우리" + 0.008*"모르" + 0.008*"매일" + 0.008*"조금" + 0.007*"사랑" + 0.007*"아프" + 0.006*"아주" + 0.006*"어느"')
(3, '0.024*"다시" + 0.015*"그대" + 0.015*"사랑" + 0.010*"아니" + 0.009*"시간" + 0.009*"모르" + 0.008*"못하" + 0.008*"마음" + 0.008*"우리" + 0.007*"채플린"')
(4, '0.013*"아니" + 0.012*"모두" + 0.010*"사랑" + 0.010*"안녕" + 0.009*"마음" + 0.009*"아무" + 0.008*"없이" + 0.008*"만나" + 0.008*"슬프" + 0.008*"이렇"')
(5, '0.020*"사랑" + 0.016*"너무" + 0.011*"모르" + 0.010*"누구" + 0.010*"비밀" + 0.009*"버리" + 0.009*"그렇" + 0.008*"아무" + 0.008*"기억" + 0.008*"마음"')


In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(LDA_model, corpus, dictionary)
pyLDAvis.display(vis)

In [ ]:
dance_id = song_df.loc[song_df['genre']=='댄스'].index

In [ ]:
for i, topic_list in enumerate(LDA_model[corpus]):
    if i in dance_id : 
      print('[{}]의 topic 비율 {}'.format(song_df['song_title'][i], topic_list))

[라일락]의 topic 비율 [(4, 0.9910911)]
[Flu]의 topic 비율 [(3, 0.9903812)]
[Celebrity]의 topic 비율 [(2, 0.9896646)]
[어푸 (Ah puh)]의 topic 비율 [(5, 0.99340963)]
[어젯밤 이야기]의 topic 비율 [(4, 0.98289865)]
[잼잼]의 topic 비율 [(4, 0.99210286)]
[Black Out]의 topic 비율 [(3, 0.9892553)]
[Zeze]의 topic 비율 [(5, 0.992778)]
[스물셋]의 topic 비율 [(4, 0.9932509)]
[Red Queen (Feat. Zion.T)]의 topic 비율 [(2, 0.9938324)]
[삐에로는 우릴 보고 웃지]의 topic 비율 [(5, 0.9881665)]
[크레파스 (드라마 '예쁜 남자' 삽입)]의 topic 비율 [(4, 0.9892328)]
[누구나 비밀은 있다 (Feat. 가인 Of Brown Eyed Girls)]의 topic 비율 [(3, 0.9888372)]
[분홍신]의 topic 비율 [(3, 0.9886861)]
[기다려]의 topic 비율 [(0, 0.9535354)]
[누구나 비밀은 있다 (Feat. 가인 Of Brown Eyed Girls)]의 topic 비율 [(3, 0.9888378)]
[분홍신]의 topic 비율 [(3, 0.9886861)]
[기다려]의 topic 비율 [(0, 0.95353323)]
[잠자는 숲 속의 왕자 (Feat. 윤상)]의 topic 비율 [(3, 0.9891373)]
[너랑 나]의 topic 비율 [(0, 0.9888259)]
[삼촌 (Feat. 이적)]의 topic 비율 [(3, 0.9898965)]
[Everything's Alright (Feat. 김현철)]의 topic 비율 [(0, 0.25649402), (3, 0.7282339)]
[좋은 날]의 topic 비율 [(5, 0.9899849)]
[미리 메리 크리스마스

In [ ]:
break_id = [0, 29, 33, 47, 65, 95, 100]

In [ ]:
for i, topic_list in enumerate(LDA_model[corpus]):
    if i in break_id : 
      print('[{}]의 topic 비율 {}'.format(song_df['song_title'][i], topic_list))

[드라마]의 topic 비율 [(3, 0.98348343)]
[이런 엔딩]의 topic 비율 [(2, 0.4329982), (3, 0.5503236)]
[마침표]의 topic 비율 [(1, 0.09205443), (3, 0.89332134)]
[사랑이 지나가면]의 topic 비율 [(3, 0.9843548)]
[(Bonus Track) Voice Mail (Korean Ver.)]의 topic 비율 [(3, 0.9892639)]
[첫 이별 그날 밤]의 topic 비율 [(3, 0.9889753)]
[기차를 타고]의 topic 비율 [(3, 0.9920505)]
